In [16]:
import pandas as pd
import langid
import re
from langdetect import detect


In [17]:
#neccessary functions
def remove_non_letters(text):
    cleaned_text = ''.join(c if c.isalpha() or c.isdigit() or c.isspace() else ' ' for c in str(text))
    return ' '.join(cleaned_text.split())

df = pd.read_csv('goodreads_nonfiction_books.csv')
df


,Book Title,Author,Average Rating,Total Rating,Publish Date,Synopsis,Review,Genres
0,The Diary of a Young Girl (Mass Market Paperback),Anne Frank,4.19,3681825,1947,Discovered in the attic in which she spent the...,Why do we write reviews?You have a lot of reas...,"['Classics', 'Nonfiction', 'Biography', 'Memoi..."
1,Sapiens: A Brief History of Humankind (Paperback),Yuval Noah Harari,4.37,1003720,2011,From a renowned historian comes a groundbreaki...,Sapiens: A Brief History of Humankind is a boo...,"['Nonfiction', 'History', 'Science', 'Philosop..."
2,Educated (Hardcover),Tara Westover,4.47,1487848,2018,Tara Westover was 17 the first time she set fo...,"On the highway below, the school bus rolls pas...","['Nonfiction', 'Memoir', 'Biography', 'Audiobo..."
3,Quiet: The Power of Introverts in a World That...,Susan Cain,4.08,430636,2012,The book that started the Quiet RevolutionAt l...,“There's zero correlation between being the be...,"['Nonfiction', 'Psychology', 'Self Help', 'Sci..."
4,The Immortal Life of Henrietta Lacks (Hardcover),Rebecca Skloot,4.11,727606,2010,"Her name was Henrietta Lacks, but scientists k...",The doorbell rang the other day and when I ans...,"['Nonfiction', 'Science', 'History', 'Biograph..."
5,Freakonomics: A Rogue Economist Explores the H...,Steven D. Levitt,4.01,846146,2005,"Which is more dangerous, a gun or a swimming p...",This was an interesting book. I say it was int...,"['Nonfiction', 'Economics', 'Business', 'Scien..."
6,Outliers: The Story of Success (Hardcover),Malcolm Gladwell,4.19,776635,2008,Learn what sets high achievers apart — from Bi...,Gladwell argues that success is tightly marrie...,"['Nonfiction', 'Psychology', 'Business', 'Self..."
7,Becoming (Hardcover),Michelle Obama,4.46,1029652,2018,In a life filled with meaning and accomplishme...,"Just to be up front, I voted for Barack in bot...","['Nonfiction', 'Memoir', 'Biography', 'Audiobo..."
8,"The Devil in the White City: Murder, Magic, an...",Erik Larson,4.00,675221,2003,Author Erik Larson imbues the incredible event...,Poor Erik Larson.He wanted to write an extensi...,"['Nonfiction', 'History', 'True Crime', 'Crime..."
9,Atomic Habits: An Easy & Proven Way to Build G...,James Clear,4.36,821265,2018,"No matter your goals, Atomic Habits offers a p...",A Book That Changed My LifeA book worth readin...,"['Nonfiction', 'Self Help', 'Psychology', 'Per..."


In [7]:
#removing all !121! which i used to seperate reviews
df['Review'] = df['Review'].str.replace("!121!", " ")

In [18]:
#remove all none letters from reviews synopsis and genres
columns_to_clean = ['Book Title','Author','Synopsis','Review','Genres']
for column in columns_to_clean:
    df[column] = df[column].apply(remove_non_letters)
    
for column in columns_to_clean:
    try:
        df[column] = df[column].str.lower()
    except:
        print ("error")


#remove all the failed data from webscraping
df_filtered = df[~df.eq("errorlololol").any(axis=1)]
df = df_filtered

df




,Book Title,Author,Average Rating,Total Rating,Publish Date,Synopsis,Review,Genres
0,the diary of a young girl mass market paperback,anne frank,4.19,3681825,1947,discovered in the attic in which she spent the...,why do we write reviews you have a lot of reas...,classics nonfiction biography memoir historica...
1,sapiens a brief history of humankind paperback,yuval noah harari,4.37,1003720,2011,from a renowned historian comes a groundbreaki...,sapiens a brief history of humankind is a book...,nonfiction history science philosophy anthropo...
2,educated hardcover,tara westover,4.47,1487848,2018,tara westover was 17 the first time she set fo...,on the highway below the school bus rolls past...,nonfiction memoir biography audiobook biograph...
3,quiet the power of introverts in a world that ...,susan cain,4.08,430636,2012,the book that started the quiet revolutionat l...,there s zero correlation between being the bes...,nonfiction psychology self help science busine...
4,the immortal life of henrietta lacks hardcover,rebecca skloot,4.11,727606,2010,her name was henrietta lacks but scientists kn...,the doorbell rang the other day and when i ans...,nonfiction science history biography medicine ...
5,freakonomics a rogue economist explores the hi...,steven d levitt,4.01,846146,2005,which is more dangerous a gun or a swimming po...,this was an interesting book i say it was inte...,nonfiction economics business science psycholo...
6,outliers the story of success hardcover,malcolm gladwell,4.19,776635,2008,learn what sets high achievers apart from bill...,gladwell argues that success is tightly marrie...,nonfiction psychology business self help socio...
7,becoming hardcover,michelle obama,4.46,1029652,2018,in a life filled with meaning and accomplishme...,just to be up front i voted for barack in both...,nonfiction memoir biography audiobook autobiog...
8,the devil in the white city murder magic and m...,erik larson,4.00,675221,2003,author erik larson imbues the incredible event...,poor erik larson he wanted to write an extensi...,nonfiction history true crime crime historical...
9,atomic habits an easy proven way to build good...,james clear,4.36,821265,2018,no matter your goals atomic habits offers a pr...,a book that changed my lifea book worth readin...,nonfiction self help psychology personal devel...


In [6]:
df.to_csv('cleaned_output_file.csv', index=False)